# 필요한 라이브러리 호출

In [1]:
from transformers import ElectraModel, ElectraTokenizer
from transformers import ElectraForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSemanticSegmentation, TrainingArguments, Trainer

import torch

import pandas as pd
from konlpy.tag import Mecab
import re
from tqdm import tqdm, tqdm_notebook

from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report

/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 데이터 불러오기 및 정제

In [2]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
interior_data = survey[['content_id', 'interior']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
interior_data['content_id'] = interior_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
interior_data = interior_data[interior_data['content_id'] != 0].reset_index().drop(columns=['index'])
interior_data['content_id'] = interior_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(interior_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
interior_content = pd.merge(interior_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
interior_content['interior'] = interior_content['interior'].astype(int)
interior_content

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_48442/1311105383.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interior_data['content_id'] = interior_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


,content_id,interior,content
0,361617,0,오류장치 ㅠㅠ\n.\n.\n옵으로 다시 참여\n그 땐 문제도 못 풀었지만 100방 ...
1,424772,0,390. 조도 무슨일이야...
2,430139,0,4인. 309번째 심연으로 떠납니다
3,431064,0,4인. 극악의 조도. 그게 다.
4,432520,0,"조도가 낮은게 아니라 없는 수준이라고 하던데, 확실히 느꼈다.. 조도도 낮은데 관찰..."
...,...,...,...
5018,425802,0,2021. 12. 26. (3인) 각자 1인분씩 하고 뿌듯했던 테마^_^
5019,428828,1,첫방 디버프로 초반에 절어버림..ㅋㅋ 볼것도 못보고 힌트썼는데 잘꾸며놓았다 ㅎㅎ 살...
5020,428930,0,#5 - 2인 - 볼륨에 압도됨 - 히터도 방마다 빵빵함 - 다른 테마도 궁금해짐
5021,429009,0,"265, 2인, 장치...ㅋㅋㅋㅋㅋㅋ 그걸 왜 몰랐냐... 장치에 속지 말자"


In [3]:
# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

interior_content['content'] = interior_content['content'].apply(cleaned_content)

final_df = interior_content[['content', 'interior']]
final_df

,content,interior
0,오류장치 . . . 옵으로 다시 참여. 그 땐 문제도 못 풀었지만 100방 이상 쌓...,0
1,390. 조도 무슨일이야.,0
2,4인. 309번째 심연으로 떠납니다,0
3,4인. 극악의 조도. 그게 다.,0
4,"조도가 낮은게 아니라 없는 수준이라고 하던데, 확실히 느꼈다. 조도도 낮은데 관찰력...",0
...,...,...
5018,2021. 12. 26. (3인) 각자 1인분씩 하고 뿌듯했던 테마.,0
5019,첫방 디버프로 초반에 절어버림. 볼것도 못보고 힌트썼는데 잘꾸며놓았다 . 살짝 장치...,1
5020,#5 - 2인 - 볼륨에 압도됨 - 히터도 방마다 빵빵함 - 다른 테마도 궁금해짐,0
5021,"265, 2인, 장치. 그걸 왜 몰랐냐. 장치에 속지 말자",0


In [4]:
# 라벨별 개수 확인
final_df['interior'].value_counts()

interior
 0    4548
 1     450
-1      25
Name: count, dtype: int64

# modeling을 위해 데이터 분리

In [5]:
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['interior'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['interior'].value_counts())

train_data interior
 0    3639
 1     359
-1      20
Name: count, dtype: int64

 test_data interior
 0    899
 1    100
-1      6
Name: count, dtype: int64


In [6]:
# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)

# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))

# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

중복 제거 전 학습 데이터셋: 4018
중복 제거 전 테스트 데이터셋: 1005


중복 제거 후 학습 데이터셋: 4003
중복 제거 후 테스트 데이터셋: 1001


In [7]:
display(train_data.head()), display(test_data.head())

,content,interior
0,매장에서 가장 쉬웠던 테마 . . 마지막과 전 문제는 일행분의 캐리로 잘 넘어갔숨....,0
1,낯선가족보단 낫다.스토리라도 낫잖아. 띠링에이트 졸업,0
2,232. 3인 역시 믿고하는 딤개님 테마,0
3,#174. 2인 w/ DMC 입졸 4연방 중 마지막 내 정수리가 스플릿 될 뻔 ㅜ ...,0
4,#384. 2인. 살짝 어렵다했는데 문제수가 많지 않아 나름 무난히 나올 수 있었다...,0


,content,interior
0,#1415_20221128_3인. 흐음.잘 열어보지 않으면,0
1,난.바보야.,0
2,들어가자마자 커피향이 씨게 낫다 좋앗다 하지만 인테리어는 그냥그랫다. 풀정도 되는듯.,1
3,3인. 스토리가 잘 감이 안 잡힌다!,0
4,1.5/3인. 활동성 있음.추락조심,0


(None, None)

# modeling 준비

In [ ]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

________

In [ ]:
# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

In [ ]:
# 전체 content 중 최대 문장 길이 확인
content_len = []
for num in final_df['content']:
    content_len.append(len(num))
max(content_len)

In [ ]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["interior"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['interior'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './interior_model/check_point/try_2',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 10,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# train 진행
trainer.train()

# 모델 저장

In [ ]:
# trainer.save_model("./interior_model/try_3/interior_model3")
# tokenizer.save_pretrained("./interior_model/try_3/interior_tokenizer3")

# 1차 모델

In [ ]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./interior_model/interior_model1/"  
tokenizer_path = "./interior_model/interior_tokenizer1/"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

## 결과 확인

In [ ]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    if predicted_class == 0:
        pred == -1
    if predicted_class == 1:
        pred == 0
    if predicted_class == 2:
        pred == 1

    return pred, probabilities

In [ ]:
# test_data의 content를 예측하여 새로운 컬럼에 넣어 예측률 확인

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)
    
test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data

In [ ]:
print(train_data['interior'].value_counts())
print(test_data['interior'].value_counts())

In [ ]:
print(classification_report(test_data['interior'], test_data['pred']))

In [ ]:
test_data2 = test_data[test_data['interior'] != 0]
test_data2

In [ ]:
print(classification_report(test_data2['interior'], test_data2['pred']))

## 한 문장에 대한 평가 (신규 문장에 대한 평가)

In [ ]:
test = ['인테리어 최악이네', '인테리어 개쩔어', '인테리어가 안 좋지는 않았어요.', '스토리 구려', '활동성은 없는데, 인테리어는 좋네요.']

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10, '\n')

## 1차 결론
- 부정 평가가 현저히 적어, 부정적인 반응에 대한 결과를 예측하지 못하고 있다고 판단.
- 인테리어가 있다는 것도 없다는 것도 모두 긍정적인 평가를 도출 -> 실질적으로 사용이 힘들 정도의 정확도를 보여줌

### 1차 결론에 대한 해결 방안
- 에폭과 배치 사이즈 등 조절을 통해서 새롭게 학습 진행 필요
- 부정적인 학습에 대한 증가를 위해 데이터 증강 진행

### 1차 모델 기초 사항
~~~
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)
training_args = TrainingArguments(
    output_dir = './interior_model',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 15,              # 훈련 epoch 수
    per_device_train_batch_size = 4,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 16,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# 2차 모델

[달라진 점]
- 과적합을 피하기 위해 에폭 수 조절 (15 -> 10)
- 훈련 배치 크기 조절 (4 -> 16)
- 평가 배치 크기 조절 (16 -> 64)     
    => 배치 크기는 학습 속도에 영향을 준다. 보다 빠른 속도로 학습 마무리 가능

In [ ]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./interior_model/try_2/interior_model2"  
tokenizer_path = "./interior_model/try_2/interior_tokenizer2"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

## 결과 확인

In [ ]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

In [ ]:
print(train_data['interior'].value_counts())
print(test_data['interior'].value_counts())

In [ ]:
print(classification_report(test_data['interior'], test_data['pred']))

In [ ]:
test_data2 = test_data[test_data['interior'] != 0]
test_data2

In [ ]:
print(classification_report(test_data2['interior'], test_data2['pred']))

## 한 문장에 대한 평가 (신규 문장에 대한 평가)

In [ ]:
test = ['인테리어 최악이네', '인테리어 개쩔어', '인테리어가 안 좋지는 않았어요.', '스토리 구려', '활동성은 없는데, 인테리어는 좋네요.']

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10, '\n')

## 2차 결론
- 1차에 비해서 중립 결과에 대해서 잘 뽑아줌
- 다만, 아직도 부정 표현에 대한 학습이 되지 않는 것으로 확인됨

### 2차 결론에 대한 해결 방안
- 데이터 증강을 통해 부정 표현이 충분히 학습될 수 있도록 진행

### 2차 모델 기초 사항
~~~
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)
training_args = TrainingArguments(
    output_dir = './interior_model/check_point/try_2',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 10,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

TrainOutput(global_step=2510, training_loss=0.026810125591033008, metrics={'train_runtime': 447.08, 'train_samples_per_second': 89.537, 'train_steps_per_second': 5.614, 'train_loss': 0.026810125591033008, 'epoch': 10.0})

# 3차 모델

[달라진 점]
- 부정 표현 학습을 위해 부정어 데이터 증강

** 그 외의 조건은 두 번째 모델과 똑같이 진행

In [15]:
# 부정 표현 학습을 위해 부정어 데이터 증강
new_data = [('인테리어 최악', -1), ('인테리어 최악은 아님', 1), ('인테리어는 최악이다.', -1), ('인테리어 디자인 정말 최악 중에 최악이야.', -1),
('인테리어 별로임', -1), ('인테리어 별로는 아니야.', 1), ('인테리어 개별로', -1),
('인테리어 안 좋음', -1), ('인테리어 안 좋지 않음', 1), ('인테리어 개좋음', 1), ('인테리어 개쩐다', 1), ('인테리어 쩐다', 1),
('인테리어 불호', -1), ('인테리어 극호', 1), ('인테리어 개빡침', -1),
('인테리어 불유쾌함', -1), ('인테리어 안 유쾌함', -1), ('인테리어 극혐', -1), ('인테리어 유쾌하지 않음', -1),
('인테리어 천상임', 1), ('인테리어 생각보다 괜찮음', 1), ('인테리어 단순함', -1), 
('인테리어 돈 아까움', -1), ('인테리어 그닥....', -1), ('인테리어 억지스러움', -1),
('노후화', -1), ('노후화 대박', -1), ('노후화 쩐다', -1), ('노후화 너무 심함', -1),
('이것도 인테리어라고 꾸며놓은 건가?', -1), ('무슨 생각으로 만든 인테리어인지 모르겠따.', -1),
('색상 조합이 안 좋지 않아 전체적으로 어수선함', -1), ('디자인이 불호를 일으켜 공간이 쾌적하지 않음', -1),
('인테리어가 개빡침을 일으키는 수준이다', -1), ('인테리어 스타일이 너무 특이해서 별로임', -1), ('인테리어가 지나치게 다양한 스타일을 혼합하여 일관성이 없음', -1),
('인테리어가 공간을 혼란스럽게 만들어 퍼즐 해결이 더 어려웠다', -1), ('퍼즐을 푸는 도중에도 인테리어가 집중을 방해했다', -1),('방탈출 공간의 디자인이 지나치게 복잡해서 불편함을 느꼈다', -1),
('방탈출에서의 인테리어는 혼란스럽게 설계돼 어려움을 더 키웠다', -1), ('퍼즐 해결에 집중해야 하는데, 난해한 인테리어 때문에 쉽게 흐트러졌다', -1), ('인테리어 디자인이 퍼즐 풀이를 방해하여 쾌적한 경험을 막았다', -1),
]
new_df = pd.DataFrame(new_data, columns=['content', 'interior'])
train_data = pd.concat([train_data, new_df])
train_data

,content,interior
0,매장에서 가장 쉬웠던 테마 . . 마지막과 전 문제는 일행분의 캐리로 잘 넘어갔숨....,0
1,낯선가족보단 낫다.스토리라도 낫잖아. 띠링에이트 졸업,0
2,232. 3인 역시 믿고하는 딤개님 테마,0
3,#174. 2인 w/ DMC 입졸 4연방 중 마지막 내 정수리가 스플릿 될 뻔 ㅜ ...,0
4,#384. 2인. 살짝 어렵다했는데 문제수가 많지 않아 나름 무난히 나올 수 있었다...,0
...,...,...
37,퍼즐을 푸는 도중에도 인테리어가 집중을 방해했다,-1
38,방탈출 공간의 디자인이 지나치게 복잡해서 불편함을 느꼈다,-1
39,방탈출에서의 인테리어는 혼란스럽게 설계돼 어려움을 더 키웠다,-1
40,"퍼즐 해결에 집중해야 하는데, 난해한 인테리어 때문에 쉽게 흐트러졌다",-1


In [16]:
new_test_df = [('방탈출 공간의 인테리어는 과도하게 혼잡해 퍼즐 해결이 더욱 어려웠다', -1), ('가구의 선택이 퍼즐과 어울리지 않아 감각적인 불편함을 느꼈다', -1),
('디자인이 복잡해서 방탈출에서 효과적으로 협력하는 데 어려움을 겪었다', -1), ('퍼즐 해결을 위한 힌트가 인테리어에 묻혀서 찾기가 까다로웠다', -1), ('방탈출 공간의 가구가 공간을 지루하게 만들어 흥미를 잃었다', -1)
]
new_test_df = pd.DataFrame(new_test_df, columns=['content', 'interior'])
test_data = pd.concat([test_data, new_df])
test_data

,content,interior,pred
0,#1415_20221128_3인. 흐음.잘 열어보지 않으면,0,0.0
1,난.바보야.,0,0.0
2,들어가자마자 커피향이 씨게 낫다 좋앗다 하지만 인테리어는 그냥그랫다. 풀정도 되는듯.,1,1.0
3,3인. 스토리가 잘 감이 안 잡힌다!,0,0.0
4,1.5/3인. 활동성 있음.추락조심,0,0.0
...,...,...,...
37,퍼즐을 푸는 도중에도 인테리어가 집중을 방해했다,-1,NaN
38,방탈출 공간의 디자인이 지나치게 복잡해서 불편함을 느꼈다,-1,NaN
39,방탈출에서의 인테리어는 혼란스럽게 설계돼 어려움을 더 키웠다,-1,NaN
40,"퍼즐 해결에 집중해야 하는데, 난해한 인테리어 때문에 쉽게 흐트러졌다",-1,NaN


In [ ]:
# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

In [ ]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["interior"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['interior'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './interior_model/check_point/try_3',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 10,              # 훈련 epoch 수
    per_device_train_batch_size = 8,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 32,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# train 진행
trainer.train()

In [ ]:
trainer.save_model("./interior_model/try_3/interior_model3")
tokenizer.save_pretrained("./interior_model/try_3/interior_tokenizer3")

In [8]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./interior_model/try_3/interior_model3"  
tokenizer_path = "./interior_model/try_3/interior_tokenizer3"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 결과 확인

In [17]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

100%|██████████| 1043/1043 [00:12<00:00, 85.78it/s]


,content,interior,pred
0,#1415_20221128_3인. 흐음.잘 열어보지 않으면,0,0
1,난.바보야.,0,0
2,들어가자마자 커피향이 씨게 낫다 좋앗다 하지만 인테리어는 그냥그랫다. 풀정도 되는듯.,1,1
3,3인. 스토리가 잘 감이 안 잡힌다!,0,0
4,1.5/3인. 활동성 있음.추락조심,0,0
...,...,...,...
37,퍼즐을 푸는 도중에도 인테리어가 집중을 방해했다,-1,1
38,방탈출 공간의 디자인이 지나치게 복잡해서 불편함을 느꼈다,-1,1
39,방탈출에서의 인테리어는 혼란스럽게 설계돼 어려움을 더 키웠다,-1,1
40,"퍼즐 해결에 집중해야 하는데, 난해한 인테리어 때문에 쉽게 흐트러졌다",-1,1


In [18]:
print(train_data['interior'].value_counts())
print(test_data['interior'].value_counts())

interior
 0    3624
 1     368
-1      53
Name: count, dtype: int64
interior
 0    895
 1    109
-1     39
Name: count, dtype: int64


In [19]:
print(classification_report(test_data['interior'], test_data['pred']))

              precision    recall  f1-score   support

          -1       0.83      0.38      0.53        39
           0       0.99      0.99      0.99       895
           1       0.76      0.93      0.83       109

    accuracy                           0.96      1043
   macro avg       0.86      0.77      0.78      1043
weighted avg       0.96      0.96      0.96      1043



In [20]:
test_data2 = test_data[test_data['interior'] != 0]
test_data2

,content,interior,pred
2,들어가자마자 커피향이 씨게 낫다 좋앗다 하지만 인테리어는 그냥그랫다. 풀정도 되는듯.,1,1
6,인테리어 향기 굿 문제 전반적으로 쉬운 문제였던거 같은데 내 능지가 얼마나 낮은지 ...,1,1
9,인테리러가 아름답네요. 카페 차리셔도 될듯.,1,1
10,2인 흥미롭게 잠수함 잘 구현 꽃길이나 세팅미스,1,0
11,"#2인. 문제가 쉽고 재밌고, 인테리어가 이쁘다. 가볍게 즐길 수 있는 테마이고. ...",1,1
...,...,...,...
37,퍼즐을 푸는 도중에도 인테리어가 집중을 방해했다,-1,1
38,방탈출 공간의 디자인이 지나치게 복잡해서 불편함을 느꼈다,-1,1
39,방탈출에서의 인테리어는 혼란스럽게 설계돼 어려움을 더 키웠다,-1,1
40,"퍼즐 해결에 집중해야 하는데, 난해한 인테리어 때문에 쉽게 흐트러졌다",-1,1


In [21]:
print(classification_report(test_data2['interior'], test_data2['pred']))

              precision    recall  f1-score   support

          -1       0.83      0.38      0.53        39
           0       0.00      0.00      0.00         0
           1       0.81      0.93      0.86       109

    accuracy                           0.78       148
   macro avg       0.55      0.44      0.46       148
weighted avg       0.81      0.78      0.77       148



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

## 한 문장에 대한 평가 (신규 문장에 대한 평가)

In [22]:
test = ['인테리어 최악이네', '인테리어 개쩔어', '인테리어가 안 좋지는 않았어요.', '스토리 구려', '활동성은 없는데, 인테리어는 좋네요.']

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10, '\n')

문장: 인테리어 최악이네, 평가: 0
---------- 

문장: 인테리어 개쩔어, 평가: 0
---------- 

문장: 인테리어가 안 좋지는 않았어요., 평가: 2
---------- 

문장: 스토리 구려, 평가: 1
---------- 

문장: 활동성은 없는데, 인테리어는 좋네요., 평가: 2
---------- 



## 3차 결론
### 확인
- 1차와 2차에 비해서 확실히 부정 표현에 대한 학습이 잘 되는 것으로 확인됨
- 다만, 부정 표현 학습과 더불어 긍정 평가도 정확도가 떨어진다는 단점이 있음
- 해당 부분은 이후 추가적으로 고려해봐야 할 것 같음

### 3차 모델 기초 사항
~~~
# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './interior_model/check_point/try_3',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 10,              # 훈련 epoch 수
    per_device_train_batch_size = 8,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 32,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# 4차

In [2]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
interior_data = survey[['content_id', 'interior']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
interior_data['content_id'] = interior_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
interior_data = interior_data[interior_data['content_id'] != 0].reset_index().drop(columns=['index'])
interior_data['content_id'] = interior_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(interior_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
interior_content = pd.merge(interior_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
interior_content['interior'] = interior_content['interior'].astype(int)

final_df = interior_content[['content', 'interior']]

# 부정 표현 학습을 위해 부정어 데이터 증강
new_data = [('인테리어 최악', -1), ('인테리어 최악은 아님', 1), ('인테리어는 최악이다.', -1), ('인테리어 디자인 정말 최악 중에 최악이야.', -1),
('인테리어 별로임', -1), ('인테리어 별로는 아니야.', 1), ('인테리어 개별로', -1),
('인테리어 안 좋음', -1), ('인테리어 안 좋지 않음', 1), ('인테리어 개좋음', 1), ('인테리어 개쩐다', 1), ('인테리어 쩐다', 1),
('인테리어 불호', -1), ('인테리어 극호', 1), ('인테리어 개빡침', -1),
('인테리어 불유쾌함', -1), ('인테리어 안 유쾌함', -1), ('인테리어 극혐', -1), ('인테리어 유쾌하지 않음', -1),
('인테리어 천상임', 1), ('인테리어 생각보다 괜찮음', 1), ('인테리어 단순함', -1), 
('인테리어 돈 아까움', -1), ('인테리어 그닥....', -1), ('인테리어 억지스러움', -1),
('노후화', -1), ('노후화 대박', -1), ('노후화 쩐다', -1), ('노후화 너무 심함', -1),
('이것도 인테리어라고 꾸며놓은 건가?', -1), ('무슨 생각으로 만든 인테리어인지 모르겠따.', -1),
('색상 조합이 안 좋지 않아 전체적으로 어수선함', -1), ('디자인이 불호를 일으켜 공간이 쾌적하지 않음', -1),
('인테리어가 개빡침을 일으키는 수준이다', -1), ('인테리어 스타일이 너무 특이해서 별로임', -1), ('인테리어가 지나치게 다양한 스타일을 혼합하여 일관성이 없음', -1),
('인테리어가 공간을 혼란스럽게 만들어 퍼즐 해결이 더 어려웠다', -1), ('퍼즐을 푸는 도중에도 인테리어가 집중을 방해했다', -1),('방탈출 공간의 디자인이 지나치게 복잡해서 불편함을 느꼈다', -1),
('방탈출에서의 인테리어는 혼란스럽게 설계돼 어려움을 더 키웠다', -1), ('퍼즐 해결에 집중해야 하는데, 난해한 인테리어 때문에 쉽게 흐트러졌다', -1), ('인테리어 디자인이 퍼즐 풀이를 방해하여 쾌적한 경험을 막았다', -1),
('방탈출 공간의 인테리어는 과도하게 혼잡해 퍼즐 해결이 더욱 어려웠다', -1), ('가구의 선택이 퍼즐과 어울리지 않아 감각적인 불편함을 느꼈다', -1),
('디자인이 복잡해서 방탈출에서 효과적으로 협력하는 데 어려움을 겪었다', -1), ('퍼즐 해결을 위한 힌트가 인테리어에 묻혀서 찾기가 까다로웠다', -1), ('방탈출 공간의 가구가 공간을 지루하게 만들어 흥미를 잃었다', -1)
]
new_df = pd.DataFrame(new_data, columns=['content', 'interior'])
final_df = pd.concat([final_df, new_df])
final_df

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

final_df['content'] = final_df['content'].apply(cleaned_content)



from kiwipiepy import Kiwi
kiwi = Kiwi()

def kiwi_clean(text):
    get_kiwi_pos = ['NNG', 'NP', 'NNP', 'MM', 'VV', 'VV-I', 'VV-R', 'VA', 'VA-I', 'VA-R', 'VCP', 'VCN', 'MAG', 'MAJ', 'XR']
    kiwi_lem = []
    for word in kiwi.tokenize(text):
        if word.tag in get_kiwi_pos:
            kiwi_lem.append(word.lemma)
    return ' '.join(kiwi_lem)

final_df['content'] = final_df['content'].apply(kiwi_clean)

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_71698/977129007.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interior_data['content_id'] = interior_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


In [3]:
# 라벨별 개수 확인
print(final_df['interior'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['interior'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['interior'].value_counts())
print('\n')
# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)

# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))
print('\n')
# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

interior
 0    4548
 1     459
-1      63
Name: count, dtype: int64


train_data interior
 0    3646
 1     362
-1      48
Name: count, dtype: int64

 test_data interior
 0    870
 1     91
-1      6
Name: count, dtype: int64


중복 제거 전 학습 데이터셋: 4056
중복 제거 전 테스트 데이터셋: 967




중복 제거 후 학습 데이터셋: 3953
중복 제거 후 테스트 데이터셋: 951


In [10]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가 
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(35080, 128)

In [11]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["interior"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['interior'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './interior_model/check_point/try_4',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 30,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [12]:
# train 진행
trainer.train()

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_69248/3586665111.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
                                                   
 30%|███       | 2239/7350 [10:46<15:23,  5.54it/s]

{'loss': 0.2242, 'learning_rate': 4.663978494623656e-05, 'epoch': 2.02}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_69248/3586665111.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
                                                   
 30%|███       | 2239/7350 [12:14<15:23,  5.54it/s]

{'loss': 0.0947, 'learning_rate': 4.327956989247312e-05, 'epoch': 4.03}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_69248/3586665111.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
                                                   
 30%|███       | 2239/7350 [13:43<15:23,  5.54it/s]

...

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_69248/3586665111.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
                                                   
 30%|███       | 2239/7350 [29:56<15:23,  5.54it/s]

{'loss': 0.0232, 'learning_rate': 2.9569892473118283e-06, 'epoch': 28.23}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_69248/3586665111.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
                                                   
100%|██████████| 7440/7440 [21:55<00:00,  5.66it/s]

{'train_runtime': 1315.3735, 'train_samples_per_second': 90.157, 'train_steps_per_second': 5.656, 'train_loss': 0.06545364984902002, 'epoch': 30.0}


TrainOutput(global_step=7440, training_loss=0.06545364984902002, metrics={'train_runtime': 1315.3735, 'train_samples_per_second': 90.157, 'train_steps_per_second': 5.656, 'train_loss': 0.06545364984902002, 'epoch': 30.0})

In [13]:
trainer.save_model("./interior_model/try_4/interior_model4")
tokenizer.save_pretrained("./interior_model/try_4/interior_tokenizer4")

('./interior_model/try_4/interior_tokenizer4/tokenizer_config.json',
 './interior_model/try_4/interior_tokenizer4/special_tokens_map.json',
 './interior_model/try_4/interior_tokenizer4/vocab.txt',
 './interior_model/try_4/interior_tokenizer4/added_tokens.json')

In [4]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./interior_model/try_4/interior_model4"
tokenizer_path = "./interior_model/try_4/interior_tokenizer4"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    text = kiwi_clean(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

100%|██████████| 951/951 [00:11<00:00, 85.31it/s]


,content,interior,pred
0,아기자기 테마 살짝 노후 느낌 있다 그렇다 좋다 테마,0,0
1,중간 재밌다 마지막 이 테마 뭐 이다 생각,0,0
2,아기자기 귀엽다 인태리어 테마,1,1
3,순간 테스터 가다 알다 장치 오류 빈번하다 일어나다 위험 보이다 장치 있다 테마 덜...,-1,1
4,첫날 가다 다행히 장치 오류 없다 엄청 일찍 가다 그런 이다 장치 오류 없다 가정 ...,0,0
...,...,...,...
961,나 너무 어렵다 방탈짬바 있다 추천,0,0
962,인 각자 하다 뿌듯하다 테마,0,0
963,첫 디버프로 초반 절다 보다 못 보다 힌트 쓰다 잘 꾸미다 살짝 장치 오류 아쉽다,1,0
964,인 볼륨 압도 히터 방 빵빵하다 다른 테마 궁금,0,0


In [10]:
print('0포함', '\n', classification_report(test_data['interior'], test_data['pred']))
test_data2 = test_data[test_data['interior'] != 0]
print('0 제외', '\n', classification_report(test_data2['interior'], test_data2['pred']))

test = ['인테리어 최악이네', '인테리어 개쩔어', '인테리어가 안 좋지는 않았어요.', '스토리 구려', '활동성은 없는데, 인테리어는 좋네요.']

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10, '\n')

0포함 
               precision    recall  f1-score   support

          -1       1.00      0.17      0.29         6
           0       1.00      0.99      0.99       854
           1       0.86      0.96      0.91        91

    accuracy                           0.98       951
   macro avg       0.95      0.70      0.73       951
weighted avg       0.98      0.98      0.98       951

0 제외 
               precision    recall  f1-score   support

          -1       1.00      0.17      0.29         6
           0       0.00      0.00      0.00         0
           1       0.95      0.96      0.95        91

    accuracy                           0.91        97
   macro avg       0.65      0.37      0.41        97
weighted avg       0.95      0.91      0.91        97

문장: 인테리어 최악이네, 평가: 0
---------- 

문장: 인테리어 개쩔어, 평가: 2
---------- 

문장: 인테리어가 안 좋지는 않았어요., 평가: 0
---------- 

문장: 스토리 구려, 평가: 1
---------- 

문장: 활동성은 없는데, 인테리어는 좋네요., 평가: 2
---------- 



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)